In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-superstore/cleaned_superstore.csv


In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv("/kaggle/input/cleaned-superstore/cleaned_superstore.csv")

In [6]:
df.head()
df.shape
df.dtypes


row_id             int64
order_id          object
order_date        object
ship_date         object
ship_mode         object
customer_id       object
customer_name     object
segment           object
country           object
city              object
state             object
postal_code        int64
region            object
product_id        object
category          object
sub-category      object
product_name      object
sales            float64
quantity           int64
discount         float64
profit           float64
dtype: object

## Feature Engineering Summary
- Date-based features added
- Business metrics derived
- Categorical normalization applied

In [7]:
df["order_year"] = pd.to_datetime(df["order_date"]).dt.year
df["order_month"] = pd.to_datetime(df["order_date"]).dt.month
df["order_quarter"] = pd.to_datetime(df["order_date"]).dt.to_period("Q").astype(str)
df["order_dayofweek"] = pd.to_datetime(df["order_date"]).dt.day_name()

In [8]:
df[["order_date", "order_year", "order_month", "order_quarter"]].head()

,order_date,order_year,order_month,order_quarter
0,2013-11-09,2013,11,2013Q4
1,2013-06-13,2013,6,2013Q2
2,2012-10-11,2012,10,2012Q4
3,2011-06-09,2011,6,2011Q2
4,2011-06-09,2011,6,2011Q2


- Business metrics derived

In [9]:
df["profit_margin"] = df["profit"] / df["sales"]

In [10]:
df["ship_delay_days"] = (
    pd.to_datetime(df["ship_date"]) -
    pd.to_datetime(df["order_date"])
).dt.days

In [11]:
df[["sales", "profit", "profit_margin", "ship_delay_days"]].describe()

,sales,profit,profit_margin,ship_delay_days
count,8827.000000,8827.000000,8827.000000,8827.000000
mean,92.864853,11.198644,0.123174,3.967260
std,114.045078,49.066101,0.485676,1.745459
min,0.444000,-1181.282400,-2.750000,0.000000
25%,15.008000,1.702400,0.075000,3.000000
50%,40.880000,7.437600,0.280000,4.000000
75%,124.225000,21.335400,0.375000,5.000000
max,498.260000,240.859500,0.500000,7.000000


Categorical normalization applied

In [12]:
categorical_cols = [
    "region",
    "segment",
    "category",
    "sub-category",
    "ship_mode"
]
for col in categorical_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.lower()
    )

In [13]:
df["region"].unique()

array(['south', 'west', 'central', 'east'], dtype=object)

## GenAI Relevance

- Aggregations → chunk-level embeddings

- Business metrics → prompt grounding

- Feature engineering → semantic signal

In [14]:
sales_by_region = (
    df.groupby("region")
      .agg(
          total_sales=("sales", "sum"),
          avg_profit=("profit", "mean"),
          order_count=("order_id", "nunique")
      )
      .reset_index()
)

In [15]:
monthly_sales = (
    df.groupby(["order_year", "order_month"])
      .agg(total_sales=("sales", "sum"))
      .reset_index()
)

In [16]:
top_products = (
    df.groupby("product_name")
      .agg(total_sales=("sales", "sum"))
      .sort_values("total_sales", ascending=False)
      .head(10)
)

In [17]:
top_products

,total_sales
product_name,
KI Adjustable-Height Table,4036.761
"Global Wood Trimmed Manager's Task Chair, Khaki",3621.004
"Situations Contoured Folding Chairs, 4/Set",2959.866
"Global High-Back Leather Tilter, Burgundy",2841.069
Nortel Meridian M3904 Professional Digital phone,2802.618
Ibico Hi-Tech Manual Binding System,2653.413
Fellowes Officeware Wire Shelving,2515.240
Easy-staple paper,2504.192
Logitech Gaming G510s - Keyboard,2362.722


In [21]:
sales_by_region.to_csv("/kaggle/working/sales_by_region.csv", index=False)
monthly_sales.to_csv("/kaggle/working/monthly_sales.csv", index=False)
top_products.to_csv("/kaggle/working/top_products.csv")

In [22]:
import os
os.listdir("/kaggle/working/")

['sales_by_region.csv',
 'monthly_sales.csv',
 'top_products.csv',
 '.virtual_documents']